In [1]:
import numpy as np 
import pandas as pd
import os
import scipy.stats as ss
import matplotlib.pyplot as plt
#import tensorflow as tf
import sklearn
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from scipy.stats import f_oneway
import pandas as pd
import datetime
#from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [5]:
# 데이터 불러오기
past_sales = pd.read_csv('rossmann.csv',low_memory=False)
store = pd.read_csv('store.csv',low_memory=False)

# 데이터 병합
data = pd.merge(left = past_sales,right = store, on = 'Store', how = 'right')

data = pd.get_dummies(data, columns=['StateHoliday', 'StoreType', 'Assortment'])

# 날짜
data['Date'] = pd.to_datetime(data['Date'], format="%Y-%m-%d")
data['Month']=pd.DatetimeIndex(data.Date).month
data['Day']=pd.DatetimeIndex(data.Date).day
data = data.drop(['Date'],axis=1)

#결측값 중간값 

data['CompetitionDistance'].fillna(data['CompetitionDistance'].median(), inplace=True)
data['CompetitionOpenSinceMonth'].fillna(data['CompetitionOpenSinceMonth'].mode()[0], inplace=True)
data['CompetitionOpenSinceYear'].fillna(data['CompetitionOpenSinceYear'].mode()[0], inplace=True)
promo_cols = ['Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']
data[promo_cols] = data[promo_cols].fillna(0)




# 데이터 가공 : PromoInterval
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
for month in months:
    data['PromoInterval_' + month] = data['PromoInterval'].apply(lambda x: 1 if month in str(x) else 0)
data.drop('PromoInterval',axis=1,inplace=True)

# 데이터 가공 : StateHoliday
scaler = MinMaxScaler()
num_cols = ['CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2SinceWeek', 'Promo2SinceYear']

data[num_cols] = scaler.fit_transform(data[num_cols])

x_train = data.drop(['Sales','Customers'],axis=1)
y_train = data['Sales']



data

In [168]:
target_col = 'Sales'
features_cols = data.columns.drop(target_col)        
features = data[features_cols]
target = data[target_col]
target = pd.DataFrame(target.values.reshape(-1,1))
features = features.values
target = target.values

In [150]:
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.4, shuffle=True)
x_test,x_valid,y_test,y_valid = train_test_split(x_test, y_test, test_size=0.5, shuffle=False)
y_train.shape

(610325, 1)

In [151]:
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
x_valid = x_valid.astype(np.float32)
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)
y_valid = y_valid.astype(np.float32)
print('train_data의 row : %s, column : %s'%(x_train.shape[0],x_train.shape[1]))
print('test_data의 row : %s, column : %s'%(x_test.shape[0],x_test.shape[1]))
print('val_data의 row : %s, column : %s'%(x_valid.shape[0],x_valid.shape[1]))
y_train

train_data의 row : 610325, column : 35
test_data의 row : 203442, column : 35
val_data의 row : 203442, column : 35


array([[7690.],
       [8431.],
       [   0.],
       ...,
       [7920.],
       [2687.],
       [3303.]], dtype=float32)

In [152]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256,input_dim = x_train.shape[1],activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dense(256,activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dense(256,activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dense(128,activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dense(1))

optimizer = tf.keras.optimizers.Adam(clipnorm=1.0)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model_path = 'C:/project/rossmann-store-sales'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=model_path , monitor='val_mean_absolute_error', verbose=1, save_best_only=True)
# early_stopping = EarlyStopping(monitor='val_loss', patience=6)

model.summary()
data.isnull().any()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_81 (Dense)             (None, 256)               9216      
_________________________________________________________________
dropout_65 (Dropout)         (None, 256)               0         
_________________________________________________________________
batch_normalization_65 (Batc (None, 256)               1024      
_________________________________________________________________
dense_82 (Dense)             (None, 256)               65792     
_________________________________________________________________
dropout_66 (Dropout)         (None, 256)               0         
_________________________________________________________________
batch_normalization_66 (Batc (None, 256)               1024      
_________________________________________________________________
dense_83 (Dense)             (None, 256)             

Store                False
DayOfWeek            False
Sales                False
Customers            False
Open                 False
Promo                False
SchoolHoliday        False
Promo2               False
StateHoliday_0       False
StateHoliday_0       False
StateHoliday_a       False
StateHoliday_b       False
StateHoliday_c       False
StoreType_a          False
StoreType_b          False
StoreType_c          False
StoreType_d          False
Assortment_a         False
Assortment_b         False
Assortment_c         False
Month                False
Day                  False
PromoInterval_Jan    False
PromoInterval_Feb    False
PromoInterval_Mar    False
PromoInterval_Apr    False
PromoInterval_May    False
PromoInterval_Jun    False
PromoInterval_Jul    False
PromoInterval_Aug    False
PromoInterval_Sep    False
PromoInterval_Oct    False
PromoInterval_Nov    False
PromoInterval_Dec    False
dtype: bool

In [153]:
early_stopping = EarlyStopping()

start = datetime.datetime.now()

history = model.fit(x_train, y_train,
                    validation_data=(x_valid, y_valid),
                    batch_size=64,
                    epochs=5,
)

end = datetime.datetime.now()
time = end-start
print("처리시간 : ", time)


Epoch 1/5
9537/9537 [==============================] - 30s 3ms/step - loss: 6.8809e-04 - accuracy: 0.1115 - val_loss: 6.8770e-04 - val_accuracy: 0.1638
Epoch 2/5
9537/9537 [==============================] - 28s 3ms/step - loss: 6.8864e-04 - accuracy: 0.1126 - val_loss: 6.8770e-04 - val_accuracy: 0.1618
Epoch 3/5
9537/9537 [==============================] - 28s 3ms/step - loss: 6.8833e-04 - accuracy: 0.1126 - val_loss: 6.8770e-04 - val_accuracy: 0.1629
Epoch 4/5
9537/9537 [==============================] - 28s 3ms/step - loss: 6.8881e-04 - accuracy: 0.1123 - val_loss: 6.8770e-04 - val_accuracy: 0.1626
Epoch 5/5
9537/9537 [==============================] - 28s 3ms/step - loss: 6.8869e-04 - accuracy: 0.1123 - val_loss: 6.8770e-04 - val_accuracy: 0.1628
처리시간 :  0:02:20.759308


In [154]:
y_pred = model.predict(x_test)

# R2 값을 계산
r2_value = r2_score(y_test, y_pred)

print("R2 값:", r2_value)

R2 값: -2.2668517590958794
